In [3]:
# pip install gspread pandas pymongo oauth2client
from __future__ import annotations

import pprint
from collections import defaultdict
from datetime import datetime, time, timedelta

import gspread
import gspread.utils
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from pymongo import InsertOne, MongoClient, UpdateOne
from pymongo.errors import PyMongoError

from config import _config

pd.set_option('display.width', 300)


# MongoDB Stuff
connection_string = f"mongodb+srv://{_config.mongo_user}:{_config.mongo_pass}@{_config.mongo_host}/?retryWrites=true&w=majority&appName=go"
mongo_client = MongoClient(connection_string)
db = mongo_client[_config.mongo_db_name]


def get_alt_id_map(db, verbose=False):
    players_collection = db["players"]
    records = list(players_collection.find({"altids": {"$ne": []}}))
    if verbose:
        print("loaded players", len(records))

    alt_id_map = {}
    delim = "\t"
    if verbose:
        print(f"ign{delim}playfabid{delim}altids")
    for r in records:
        for altid in r["altids"]:
            alt_id_map[altid] = r["playfabid"]
            if verbose:
                print(f"{r['ign']}{delim}{r['playfabid']}{delim}{altid}")

    return alt_id_map


def run_bulk_operations(bulk_operations, collection):
    print(f"Bulk Operations To Run: {len(bulk_operations)}")
    if bulk_operations:
        result = collection.bulk_write(bulk_operations)
        print(f" - Matched:  {result.matched_count}")
        print(f" - Modified: {result.modified_count}")
        print(f" - Inserted: {result.inserted_count}")

def update_season(season):
    bulk_ops = [UpdateOne({"_id": season["_id"]}, {"$set": season})]
    run_bulk_operations(bulk_ops, seasons_collection)
    

seasons_collection = db["seasons"]
seasons = list(seasons_collection.find())
print("loaded seasons", len(seasons))

season = {}
for s in seasons:
    pprint.pp(s)
    if s["active"]:
        season = s

assert season
print("Active Season:", season['league'], season['season'])



loaded seasons 1
{'_id': ObjectId('66ab0037edc64cf2a40ce743'),
 'league': 'GO Phoenix',
 'season': 1,
 'sheetname': 'Match Results Phoenix S1',
 'url': 'https://docs.google.com/spreadsheets/d/1KqwMFmRg-BD6-BT2NcPW9_wM2vgogRCEQGq8x5DbFeI/',
 'active': True,
 'tabs': [{'name': 'Match Results', 'rowsread': 0},
          {'name': 'Screenshot Results', 'rowsread': 0}]}
Active Season: GO Phoenix 1


# Load Raw Scores


In [4]:
from __future__ import annotations

def number_to_column_letter(n):
    """Convert a column number to a spreadsheet column letter."""
    string = ""
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        string = chr(65 + remainder) + string
    return string


def get_record_count(worksheet):
    column_a_values = worksheet.col_values(1)  # Column A is the 1st column
    non_empty_values = [value for value in column_a_values if value]
    return len(non_empty_values)-1 # Subtract 1 to exclude the header row

def read_new_statbot_rows(season, tab, gsheets_credentials) -> [dict]: # type: ignore
    # Authenticate and initialize the gspread client
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    credentials = ServiceAccountCredentials.from_json_keyfile_name(gsheets_credentials, scope)  # type: ignore
    client = gspread.authorize(credentials)  # type: ignore

    # Open the Google Sheet and the specified tab
    sheet = client.open(season['sheetname'])
    worksheet = sheet.worksheet(tab['name'])

    # non-empty rows count
    record_count = get_record_count(worksheet)  

    print(f'In tab "{tab["name"]}" in sheet "{season["sheetname"]}" has:  \n - {worksheet.row_count = } \n - nonempty row count: {record_count} \n - rowsread: {tab["rowsread"]}')


    # if record_count <= tab['rowsread']:
    #     print("No new records to load")
    #     return []
    
    header_row = worksheet.row_values(1)
    column_count = len(header_row)
    start_row = tab["rowsread"]+2 # +1 for header, +1 to start at the first new row
    end_row = record_count +1 # +1 for 1-based indexing
    range = f'A{start_row}:{number_to_column_letter(column_count)}{end_row}'

    print(f'column_count: {column_count} -> {number_to_column_letter(column_count)} -> {range}')
    
    new_scores = []

    if start_row <= end_row:
        value_rows = worksheet.get_values(range)
        new_scores = [dict(zip(header_row, gspread.utils.numericise_all(row))) for row in value_rows]

    print(f'Read {len(new_scores)} new scores')
    return new_scores


def processs_statbot_scoresheet(season, records):
    # Convert records to DataFrame
    df = pd.DataFrame(records)
    # df = df[:10]
    # print(df)

    # Example of converting a datetime string column to datetime objects
    # Assuming your datetime column is named 'datetime_column'
    df["date"] = df["date"].apply(lambda x: datetime.strptime(x, "%Y/%m/%d %H:%M:%S"))

    # add columns for season and league to the front
    # df.insert(0, "season", 1)
    # df.insert(0, "league", "GO Phoenix")
    df.insert(0, "season", season['season'])
    df.insert(0, "league", season['league'])

    # rename match to game
    df.rename(columns={"match": "game"}, inplace=True)

    # split up column names so they can be nested in the mongo document
    # sub-dicts are separated by '.'
    def update_col_name(col):
        for word in "Firearm Buff Sniper Shotgun SMG RocketLauncher Pistol Assault".split():
            col = col.replace(word, word + "_")
        col = col.replace("__", "_")
        col = col.replace("_", ".")
        return col

    df.columns = [update_col_name(col) for col in df.columns]

    # move these columns ahread of the Firearm columns
    insert_position = df.columns.get_loc("damage") + 1  # type: ignore
    for col in "assists botkills botdamage".split():
        if col in df:
            column = df.pop(col)
            df.insert(insert_position, col, column)  # type: ignore

    print("Read in Columns From Spreadsheet:", [_ for _ in df.columns])
    # print(df)

    return df


def convert_to_nested_dicts(df):

    def convert_row_to_nested_dicts(record: dict) -> dict:
        nested_record = {}
        for key, value in record.items():
            keys = key.split(".")
            d = nested_record
            for subkey in keys[:-1]:
                if subkey not in d:
                    d[subkey] = {}
                d = d[subkey]
            d[keys[-1]] = value
        return nested_record

    data_dict = df.to_dict("records")
    nested_dicts = []

    for r in data_dict:
        cleaned_record = {k: v for k, v in r.items() if v not in [None, ""]}
        cleaned_record = convert_row_to_nested_dicts(cleaned_record)
        nested_dicts.append(cleaned_record)

    return nested_dicts


def load_data_to_mongo(nested_dicts):
    scores_collection = db["scores"]
    result = scores_collection.insert_many(nested_dicts)
    print(f"Acknowledged: {result.acknowledged}")
    print(f"Inserted count: {len(result.inserted_ids)}")


def load_tab_to_mongodb(season, tab, gsheets_credentials):
    sheet_name = season['sheetname']
    tab_name = tab['name']
    rowsread_before = tab['rowsread']

    print(f'\n\nLoading  sheet: "{sheet_name}"  tab: "{tab_name}"  rowsread: {rowsread_before}')

    new_records = read_new_statbot_rows(season, tab, gsheets_credentials)
    rowsread_after = rowsread_before + len(new_records)

    if not new_records:
        print("No new records to load")
        return

    df = processs_statbot_scoresheet(season, new_records)
    df["loadedfrom"] = f"{sheet_name} -- {tab_name}"

    # verify all the columns we need are present
    for col in "league season date code game team player playfabid placement kills damage".split():
        assert col in df

    nested_dicts = convert_to_nested_dicts(df)

    load_data_to_mongo(nested_dicts)

    tab['rowsread'] = rowsread_after
    update_season(season)



# Google Sheets credentials and sheet details
CREDENTIALS_FILE = "../_google_sheets_credentials.json"

for tab in season['tabs']:
    load_tab_to_mongodb(season, tab, CREDENTIALS_FILE)



Loading  sheet: "Match Results Phoenix S1"  tab: "Match Results"  rowsread: 0
In tab "Match Results" in sheet "Match Results Phoenix S1" has:  
 - worksheet.row_count = 2678 
 - nonempty row count: 2677 
 - rowsread: 0
column_count: 295 -> KI -> A2:KI2678
Read 2677 new scores
Read in Columns From Spreadsheet: ['league', 'season', 'date', 'code', 'game', 'team', 'player', 'playfabid', 'placement', 'kills', 'damage', 'botdamage', 'botkills', 'assists', 'revives', 'deaths', 'damagetaken', 'firstkill', 'Buff.Banana.count', 'Buff.Banana.mintime', 'Buff.Banana.maxtime', 'Buff.Banana.avgtime', 'Firearm.damage', 'Firearm.shotsfired', 'Firearm.shotshit', 'Firearm.headshots', 'Firearm.onehandfired', 'Firearm.onehandhit', 'Firearm.twohandfired', 'Firearm.twohandhit', 'Firearm.mindist', 'Firearm.maxdist', 'Firearm.avgdist', 'Firearm.Assault.AKM.damage', 'Firearm.Assault.AKM.shotsfired', 'Firearm.Assault.AKM.shotshit', 'Firearm.Assault.AKM.headshots', 'Firearm.Assault.AKM.onehandfired', 'Firearm.

# Load Players from Raw Scores


In [5]:
scores_collection = db["scores"]

projection = {"Buff": 0, "Firearm": 0}
records = list(scores_collection.find({"player": {"$ne": "TOTALS"}}, projection).sort("_id", 1))
print("loaded scores", len(records))


player_scores_map = defaultdict(list)
for r in records:
    player_scores_map[r["playfabid"]].append(r)
print(f"loaded {len(player_scores_map)} playfabids")


players = []
for playfabid, pscores in player_scores_map.items():

    ign_info_map = defaultdict(dict)
    teams = set()
    for score in pscores:
        ign_info = ign_info_map[score["player"]]
        if "mindate" not in ign_info:
            ign_info["counts"] = 1
            ign_info["mindate"] = score["date"]
            ign_info["maxdate"] = score["date"]
        else:
            ign_info["counts"] += 1
            ign_info["mindate"] = min(score["date"], ign_info["mindate"])
            ign_info["maxdate"] = max(score["date"], ign_info["maxdate"])

        # teams.add(score['teamname'])

    player = {}
    player["playfabid"] = playfabid

    sorted_ign_counts = sorted([(info["counts"], ign) for (ign, info) in ign_info_map.items()])
    player["ign"] = sorted_ign_counts[-1][1]

    player["altids"] = []
    player["teams"] = list(teams)
    player["ignhistory"] = ign_info_map

    players.append(player)

    # if len(sorted_ign_counts) > 1:
    #     pprint.pp(sorted_ign_counts)
    #     pprint.pp(player)


players_collection = db["players"]
records = list(players_collection.find())
print("loaded players", len(records))

player_map = {r["playfabid"]: r for r in records}

bulk_operations = []
for player in players:
    if player["playfabid"] in player_map:
        prev_player = player_map[player["playfabid"]]
        bulk_operations.append(UpdateOne({"_id": prev_player["_id"]}, {"$set": player}))
    else:
        bulk_operations.append(InsertOne(player))

run_bulk_operations(bulk_operations, players_collection)

loaded scores 2031
loaded 149 playfabids
loaded players 0
Bulk Operations To Run: 149
 - Matched:  0
 - Modified: 0
 - Inserted: 149


# Update Players Collection with Alts


In [6]:
players_collection = db["players"]
records = list(players_collection.find())
print("loaded players", len(records))

player_map = {r["playfabid"]: r for r in records}


main_to_alt = {}
main_to_alt[7335627913170368] = 8070950232971034  # SoIace
main_to_alt[6884244421666056] = 6390243121089235  # BarkleyBTYC
main_to_alt[4752641078192893] = 6431101783663547  # VIPxNiya
main_to_alt[6357185327664252] = 6546474178745784  # IrieVR
main_to_alt[9919676141437463] = 7572820000000000  # BBBxDimez
main_to_alt[24062625310049943] = 7822552054442101  # Consesa
main_to_alt[6719539931430449] = 24176678161931153  # Ballo

bulk_operations = []
for main_id, alt_id in main_to_alt.items():
    player = player_map[main_id]
    if alt_id not in player["altids"]:
        player["altids"].append(alt_id)
        bulk_operations.append(UpdateOne({"_id": player["_id"]}, {"$set": player}))


run_bulk_operations(bulk_operations, players_collection)

loaded players 149
Bulk Operations To Run: 7
 - Matched:  7
 - Modified: 7
 - Inserted: 0


# Add to Raw Scores -- teamname, roster, isfungame, atl playfabids, etc


In [7]:
# Create a MongoClient
import itertools
from collections import defaultdict

from pymongo import UpdateOne

scores_collection = db["scores"]

projection = {"Buff": 0, "Firearm": 0}
records = list(scores_collection.find({}, projection).sort("_id", 1))
print("loaded records", len(records))


def placement_points(placement):
    if placement == 1:
        return 10
    elif placement == 2:
        return 7
    elif placement == 3:
        return 5
    elif placement == 4:
        return 3
    elif placement == 5:
        return 1
    return 0


def calc_score(r, placement_share):
    score = 0.0
    score += placement_points(r["placement"]) * 1.0 / placement_share
    score += 2 * r["kills"]
    score += r["damage"] / 200.0
    return score


def team_key(r):
    return tuple(r[_] for _ in "league season code team".split())


def team_game_key(r):
    return tuple(r[_] for _ in "league season code game team".split())


def season_key(r):
    return tuple(r[_] for _ in "league season code".split())


alt_id_map = get_alt_id_map(db)

roster_map = defaultdict(dict)
season_fungame_map = defaultdict(dict)
playfabid_to_name = {}

for r in records:
    # track the roster for each team in each game
    if r["player"] != "TOTALS":

        # map alt accounts to the main playfabid
        if r["playfabid"] in alt_id_map:
            r["playfabid_orig"] = r["playfabid"]
            r["playfabid"] = alt_id_map[r["playfabid"]]

        roster_map[team_game_key(r)][r["playfabid"]] = r
        playfabid_to_name[r["playfabid"]] = r["player"]

    # track the games each season so we can rule out fun games
    season_fungame_map[season_key(r)][(r["game"])] = False

for key, fungame_map in season_fungame_map.items():
    for i, game in enumerate(sorted(fungame_map.keys())):
        if i >= 5:
            season_fungame_map[key][game] = True
            print("Fun game detected", key, "match", game)

for r in records:
    r["isfungame"] = season_fungame_map[season_key(r)][r["game"]]
    r["postprocessed"] = "v1"


# filter out fun games
fungame_records = [r for r in records if r["isfungame"] == True]
records = [r for r in records if r["isfungame"] == False]

# calculate team roster for the whole season (not just each game)
team_to_roster = defaultdict(set)
for r in records:
    roster_for_game = roster_map[team_game_key(r)]
    team_to_roster[team_key(r)].update(roster_for_game.keys())

for r in records:
    roster = team_to_roster[team_key(r)]

    r["rosterids"] = sorted(roster)  # sort the playfabids so the team name is consistent
    r["roster"] = [playfabid_to_name[_] for _ in r["rosterids"]]  # convert playfabids to names in the same order

    # test for these b/c sometimes they are not accurate
    # like if we only loaded TOTALS from screenshots
    if "teamsize" not in r:
        r["teamsize"] = len(roster)
    if "teamname" not in r:
        r["teamname"] = ", ".join(r["roster"])

    # calculate the score for the team or for the player
    # a players share of the placement points is 1 divided by the number of players on the team that game
    placement_share = 1
    if r["player"] != "TOTALS":
        roster_for_game = roster_map[team_game_key(r)]
        placement_share = len(roster_for_game)
    r["score"] = calc_score(r, placement_share)

    # print([r.get(_,"") for _ in 'code team player playfabid placement kills damage score teamsize isfungame postprocessed roster rosterids'.split()])


# Prepare the bulk operations -- 6x faster than individual updates
bulk_operations = []
for r in itertools.chain(records, fungame_records):
    bulk_operations.append(UpdateOne({"_id": r["_id"]}, {"$set": r}))

run_bulk_operations(bulk_operations, scores_collection)

loaded records 2877
Fun game detected ('GO Phoenix', 1, '238BD4') match 5
Fun game detected ('GO Phoenix', 1, 'E8E6I6') match 5
Fun game detected ('GO Phoenix', 1, 'E3OC98') match 5
Bulk Operations To Run: 2877
 - Matched:  2877
 - Modified: 2877
 - Inserted: 0


# Raw Scores --> Session Scores


In [8]:
from datetime import datetime, time, timedelta
from pymongo import InsertOne


scores_collection = db["scores"]

projection = {"Buff": 0, "Firearm": 0}

records = list(scores_collection.find({"isfungame": False, "player": "TOTALS"}, projection).sort("_id", 1))
print("loaded scores", len(records))

key_names = "league season code teamname".split()


def team_session_key(r):
    return tuple(r[_] for _ in key_names)


def session_key(r):
    return tuple(r[_] for _ in "league season code".split())


def is_win(place):
    if place == 1:
        return 1
    return 0


def round_to_previous_half_hour(dt: datetime) -> datetime:
    min_to_sub = dt.minute % 30
    rounded_time = dt - timedelta(minutes=min_to_sub, seconds=dt.second, microseconds=dt.microsecond)
    return rounded_time


session_times_map = defaultdict(list)
session_scores_map = defaultdict(list)
for r in records:
    session_scores_map[team_session_key(r)].append(r)
    session_times_map[session_key(r)].append(r["date"])

session_scores = []

for key, sscores in session_scores_map.items():
    session_times = session_times_map[session_key(sscores[0])]
    session_start = round_to_previous_half_hour(min(session_times))

    ngames = 1.0 * len(sscores)
    score_values = sorted([r["score"] for r in sscores], reverse=True)

    session_score = dict()
    for key_name in key_names + ["teamsize"]:
        session_score[key_name] = sscores[0][key_name]

    session_score["session_time"] = session_start
    session_score["min_time"] = min([r["date"] for r in sscores])
    session_score["max_time"] = max([r["date"] for r in sscores])
    session_score["games"] = int(ngames)
    session_score["wr"] = sum([is_win(r["placement"]) for r in sscores]) / ngames
    session_score["kpg"] = sum([r["kills"] for r in sscores]) / ngames
    session_score["dpg"] = sum([r["damage"] for r in sscores]) / ngames
    # session_score['apg'] = sum([r['assists'] for r in sscores]) / ngames
    # session_score['deathspg'] = sum([r['deaths'] for r in sscores]) / ngames
    # session_score['botkpg'] = sum([r['botkills'] for r in sscores]) / ngames
    # session_score['botdpg'] = sum([r['botdamage'] for r in sscores]) / ngames
    session_score["score"] = sum(score_values[:4])
    session_score["minscore"] = min(score_values)
    session_score["scores_total"] = sum(score_values)
    session_score["scores"] = [r["score"] for r in sscores]

    session_score["roster"] = sscores[0]["roster"]
    session_score["rosterids"] = sscores[0]["rosterids"]

    session_score["scoreids"] = [r["_id"] for r in sscores]

    session_scores.append((key, session_score))
    # pprint.pp(session_score)


session_scores_collection = db["session_scores"]
records = list(session_scores_collection.find())
print("loaded session scores", len(records))

player_map = {team_session_key(r): r for r in records}


bulk_operations = []
for key, session_score in session_scores:
    if key in player_map:
        previous_score = player_map[key]
        bulk_operations.append(UpdateOne({"_id": previous_score["_id"]}, {"$set": session_score}))
    else:
        bulk_operations.append(InsertOne(session_score))


run_bulk_operations(bulk_operations, session_scores_collection)

loaded scores 828
loaded session scores 0
Bulk Operations To Run: 172
 - Matched:  0
 - Modified: 0
 - Inserted: 172


# Session Scores --> Season Scores


In [9]:
session_scores_collection = db["session_scores"]

records = list(session_scores_collection.find())
print("loaded session_scores_collection", len(records))

key_names = "league season teamname".split()


def team_season_key(r):
    return tuple(r[_] for _ in key_names)


def season_key(r):
    return tuple(r[_] for _ in "league season code".split())


season_times_map = defaultdict(list)
season_scores_map = defaultdict(list)
for r in records:
    season_scores_map[team_season_key(r)].append(r)
    season_times_map[season_key(r)].append(r["min_time"])
    season_times_map[season_key(r)].append(r["max_time"])

season_scores = []

for key, sscores in season_scores_map.items():

    # make sure the session with the full roster is first
    # a roster can be incomplete b/c sometimes the roster isn't loaded if StatBot is down
    # and we only have screenshots
    sscores.sort(key=lambda r: len(r['roster']), reverse=True)
    if sscores[0]["teamname"] == "BaIIo, VIPxDogmom08, GOxPinkpwnage":
        print(sscores[0])
        print([_['roster'] for _ in sscores])

    season_times = season_times_map[season_key(sscores[0])]

    nsessions = 1.0 * len(sscores)
    ngames = 1.0 * sum([r["games"] for r in sscores])

    score_values = sorted([r["score"] for r in sscores], reverse=True)

    season_score = dict()
    for key_name in key_names + ["teamsize"]:
        season_score[key_name] = sscores[0][key_name]

    season_score["min_time"] = min(season_times)
    season_score["max_time"] = max(season_times)
    season_score["sessions"] = int(nsessions)
    season_score["games"] = int(ngames)
    season_score["wr"] = sum([r["wr"] * r["games"] for r in sscores]) / ngames
    season_score["kpg"] = sum([r["kpg"] * r["games"] for r in sscores]) / ngames
    # season_score['dpg'] = sum([r['dpg']*r['games'] for r in sscores]) / ngames
    # season_score['apg'] = sum([r['apg']*r['games'] for r in sscores]) / ngames
    # season_score['deathspg'] = sum([r['deathspg']*r['games'] for r in sscores]) / ngames
    # season_score['botkpg'] = sum([r['botkpg']*r['games'] for r in sscores]) / ngames
    # season_score['botdpg'] = sum([r['botdpg']*r['games'] for r in sscores]) / ngames
    season_score["score"] = sum(score_values[:3])
    season_score["avgscore"] = 1.0 * sum(score_values[:3]) / len(score_values[:3])
    season_score["minscore"] = min(score_values)
    season_score["scores_total"] = sum(score_values)
    season_score["scores"] = [r["score"] for r in sscores]

    
    season_score["roster"] = sscores[0]["roster"]
    season_score["rosterids"] = sscores[0]["rosterids"]

    season_score["seasonscoreids"] = [r["_id"] for r in sscores]

    season_scores.append((key, season_score))
    # pprint.pp(season_score)


season_scores_collection = db["season_scores"]
records = list(season_scores_collection.find())
print("loaded season scores", len(records))

previous_season_scores = {team_season_key(r): r for r in records}


bulk_operations = []
for key, season_score in season_scores:
    if key in previous_season_scores:
        previous_score = previous_season_scores[key]
        bulk_operations.append(UpdateOne({"_id": previous_score["_id"]}, {"$set": season_score}))
    else:
        bulk_operations.append(InsertOne(season_score))


run_bulk_operations(bulk_operations, season_scores_collection)

loaded session_scores_collection 172
{'_id': ObjectId('66b01a23d44a8b560492f43d'), 'league': 'GO Phoenix', 'season': 1, 'code': 'CC7DAO', 'teamname': 'BaIIo, VIPxDogmom08, GOxPinkpwnage', 'teamsize': 3, 'session_time': datetime.datetime(2024, 7, 13, 18, 30), 'min_time': datetime.datetime(2024, 7, 13, 18, 49, 13), 'max_time': datetime.datetime(2024, 7, 13, 19, 57, 10), 'games': 4, 'wr': 0.5, 'kpg': 17.25, 'dpg': 4210.0, 'score': 246.2, 'minscore': 36.415, 'scores_total': 246.2, 'scores': [72.655, 39.879999999999995, 36.415, 97.25], 'roster': ['BaIIo', 'VIPxDogmom08', 'GOxPinkpwnage'], 'rosterids': [6719539931430449, 6813253648695082, 6857661557591587], 'scoreids': [ObjectId('66b01a07d44a8b560492e9d7'), ObjectId('66b01a07d44a8b560492ea06'), ObjectId('66b01a07d44a8b560492ea22'), ObjectId('66b01a07d44a8b560492ea37')]}
[['BaIIo', 'VIPxDogmom08', 'GOxPinkpwnage'], ['BaIIo', 'VIPxDogmom08', 'GOxPinkpwnage'], ['BaIIo', 'VIPxDogmom08', 'GOxPinkpwnage'], ['GOxPinkpwnage']]
loaded season scores 0

# Existing Alts


In [ ]:
alt_id_map = get_alt_id_map(db)
pprint.pp(alt_id_map)

# Finding Alts


In [ ]:
import re

players_collection = db["players"]
records = list(players_collection.find())
# print('loaded players', len(records))


records.sort(key=lambda r: r["ign"])

for r in records:
    counts = [r["ignhistory"][ign]["counts"] for ign in r["ignhistory"]]
    ign2 = r["ign"]
    ign2 = re.sub("^[A-Z]{2,3}[_-x\\.~]", "", ign2)
    ign2 = re.sub("[_-x\\.~][A-Z]{2,3}$", "", ign2)

    print(f"{r['ign']}\t{r['playfabid']}\t{sum(counts)}\t{ign2}")